<a href="https://colab.research.google.com/github/VavRe/nlp-ut/blob/main/PPMI_Sentiment_Snapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pickle
import pandas as pd
import numpy as np
import string
import hazm
from tqdm import tqdm

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
COLAB_DATASET_PATH = "/content/drive/MyDrive/NLP/CA2/preprocessed.pkl"
LOCAL_DATASET_PATH = './datasets/preprocessed.pkl'
dataset = pd.read_pickle(COLAB_DATASET_PATH)

In [7]:
dataset = dataset.sample(frac=1, random_state=4)
train_data = dataset[:int(len(dataset)*0.9)]
test_data = dataset[int(len(dataset)*0.9):]
print("Dataset size: ", len(dataset))
print("Test Data size: ", len(test_data))
print("Train Data size: ", len(train_data))

Dataset size:  14000
Test Data size:  1400
Train Data size:  12600


In [8]:

vocabulary = set()
tf = dict()


def make_vocab(sentence):

    for word in sentence:
         vocabulary.add(word)


train_data["comment"].apply(make_vocab)


8756     None
5474     None
11242    None
7820     None
7909     None
         ... 
10142    None
8828     None
11554    None
11609    None
904      None
Name: comment, Length: 12600, dtype: object

In [9]:
word_index = dict()
def map_word_index(vocabulary):
    for index,word in enumerate(vocabulary):
        word_index[word] = index
        
map_word_index(vocabulary)

In [10]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
co_occurance_matrix = np.zeros([len(vocabulary),len(vocabulary)])
stopwords = hazm.stopwords_list()
for comment in tqdm(train_data["comment"], desc="Comment Loop"):
    for word in comment:
        for other_words in comment:
            if other_words != word and other_words not in string.punctuation and other_words not in stopwords:
                co_occurance_matrix[word_index[word]][word_index[other_words]] += 1

Comment Loop: 100%|██████████| 12600/12600 [00:37<00:00, 338.81it/s]


In [12]:
def check_most_cooc(word):
  largest_indices = np.argpartition(co_occurance_matrix[word_index[word]], -5)[-5:]
  for i in largest_indices:
    print(list(vocabulary)[i])

check_most_cooc("نوشابه")

گرم
ارسال
غذا
سفارش
،


In [13]:
probability_matrix = co_occurance_matrix/np.sum(co_occurance_matrix)
word_probability = np.sum(probability_matrix,1)

ppmi_matrix = np.zeros([len(vocabulary),len(vocabulary)])

# Compute the outer product of word_probability to get a matrix of shape (len(vocabulary), len(vocabulary))
# where each element (i, j) is the product of word_probability[i] and word_probability[j]
outer = np.outer(word_probability, word_probability)

# Compute the log of probability_matrix and divide it by the outer product
# Use np.maximum to ensure that all values are non-negative
ppmi_matrix = np.where(outer != 0, np.maximum(np.log(probability_matrix/outer), 0), 0)

# Set the diagonal elements to zero
np.fill_diagonal(ppmi_matrix, 0)


# ppmi_matrix = np.zeros([len(vocabulary),len(vocabulary)])
# for i in tqdm(range(len(vocabulary)), desc="PPMI Matrix"):
#     for j in range(len(vocabulary)):
#         if i != j:
#             ppmi_matrix[i][j] = max(np.log(probability_matrix[i][j]/(word_probability[i]*word_probability[j])),0)

<ipython-input-13-d6ae4b2c1ab1>:12: RuntimeWarning: divide by zero encountered in true_divide
  ppmi_matrix = np.where(outer != 0, np.maximum(np.log(probability_matrix/outer), 0), 0)
<ipython-input-13-d6ae4b2c1ab1>:12: RuntimeWarning: invalid value encountered in true_divide
  ppmi_matrix = np.where(outer != 0, np.maximum(np.log(probability_matrix/outer), 0), 0)
<ipython-input-13-d6ae4b2c1ab1>:12: RuntimeWarning: divide by zero encountered in log
  ppmi_matrix = np.where(outer != 0, np.maximum(np.log(probability_matrix/outer), 0), 0)


In [14]:
def fit_ppmi(comment):
    vectors = np.zero
    for word in comment:
        if word in vocabulary:
          vectors.append(ppmi_matrix[word_index[word]])
    return np.mean(vectors,0)


In [15]:
train_data["ppmi_vectorized"] = train_data["comment"].apply(fit_ppmi)
test_data["ppmi_vectorized"] = test_data["comment"].apply(fit_ppmi)

<ipython-input-15-40e46d49c36e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data["ppmi_vectorized"] = train_data["comment"].apply(fit_ppmi)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
<ipython-input-15-40e46d49c36e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data["ppmi_vectorized"] = test_data["comment"].apply(fit_ppmi)


In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
clf = MultinomialNB()
clf.fit(train_data["ppmi_vectorized"].to_list(),train_data["label_id"].to_list())


MultinomialNB()

In [22]:
test_X = test_data["ppmi_vectorized"].to_list()
pred = clf.predict(test_X)

classification_report(pred,test_data["label_id"].to_list())


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:185: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array = numpy.asarray(array, order=order, dtype=dtype)


ValueError: ignored

In [28]:
test = np.array(test_X)
test.shape

<ipython-input-28-7f6a1fb38aaf>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test = np.array(test_X)


(1400,)